In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import math

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
!ls "drive/My Drive/ML2020/"

 modelling_set.csv  'New Data.rar'


In [8]:
!cp -r "drive/My Drive/ML2020/modelling_set.csv" "/content"
!ls

drive  modelling_set.csv  sample_data


In [2]:
df = pd.read_csv('../1.Data Processing/new data/modelling_set.csv')
df.describe()

,MemberID,LabCount_total,LabCount_max,LabCount_min,LabCount_ave,LabCount_months,LabCount_std,DrugCount_total,DrugCount_max,DrugCount_min,...,ps8,ps9,AgeAtFirstClaim,Male,Female,MissSex,MissAge,ClaimsTruncated,TARGET,trainset
count,2.184150e+05,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,...,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.000000,218415.0,218415.000000,147473.000000,218415.000000
mean,4.993458e+07,6.471827,3.829439,2.181860,2.918800,1.655033,0.578166,10.387318,2.193531,0.962237,...,0.109869,0.073507,47.750155,0.363524,0.448078,0.188398,0.0,0.054863,0.452951,0.675196
std,2.892479e+07,7.734678,3.440373,2.507408,2.676663,1.906704,1.683426,15.721582,2.271990,1.057978,...,1.124365,0.571573,23.238375,0.481015,0.497298,0.391031,0.0,0.227714,1.573818,0.468303
min,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,2.478774e+07,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,35.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
50%,4.993677e+07,4.000000,4.000000,1.000000,2.666667,1.000000,0.000000,3.000000,2.000000,1.000000,...,0.000000,0.000000,45.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000
75%,7.515525e+07,10.000000,7.000000,3.000000,5.000000,2.000000,1.527525,14.000000,4.000000,1.000000,...,0.000000,0.000000,65.000000,1.000000,1.000000,0.000000,0.0,0.000000,0.000000,1.000000
max,9.999882e+07,111.000000,10.000000,10.000000,10.000000,12.000000,6.363961,84.000000,7.000000,7.000000,...,43.000000,37.000000,80.000000,1.000000,1.000000,1.000000,0.0,1.000000,15.000000,1.000000


In [3]:
df_train = df[df['trainset']==1]
df_target = df[df['trainset']==0]

In [4]:
#Count missing data
def count_missing(df):   
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(20))
    return missing_data

count_missing(df_train)

                     Total  Percent
trainset                 0      0.0
CharlsonIndex_ave        0      0.0
pgc3                     0      0.0
pgc2                     0      0.0
pgc1                     0      0.0
CharlsonIndex_range      0      0.0
DSFS_range               0      0.0
CharlsonIndex_std        0      0.0
CharlsonIndex_min        0      0.0
pgc5                     0      0.0
CharlsonIndex_max        0      0.0
CharlsonIndex_sum        0      0.0
DSFS_std                 0      0.0
DSFS_ave                 0      0.0
DSFS_min                 0      0.0
DSFS_max                 0      0.0
pgc4                     0      0.0
pgc6                     0      0.0
pgc23                    0      0.0
pgc15                    0      0.0


,Total,Percent
trainset,0,0.0
CharlsonIndex_ave,0,0.0
pgc3,0,0.0
pgc2,0,0.0
pgc1,0,0.0
...,...,...
sp7,0,0.0
sp8,0,0.0
sp9,0,0.0
sp10,0,0.0


In [5]:
m = (df_train.columns != 'TARGET') & (df_train.columns != 'MemberID') & (df_train.columns != 'Year') & (df_train.columns != 'trainset')
X = df_train.loc[:, m]
y = df_train['TARGET']
y.describe()

count    147473.000000
mean          0.452951
std           1.573818
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          15.000000
Name: TARGET, dtype: float64

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(X,)
X = scaler.transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [8]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    # The root mean square logarithmic error
    pred_y = pred_y.clip(min=0)
    pred_y = pred_y.clip(max=15)
    pred_y = pred_y.round(0).astype(int)
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    print ("RMSLE: %.2f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(y_test,pred_y)
    print ("R2 value: %.2f" % r2)
    from sklearn.metrics import mean_squared_log_error
    err = math.sqrt(mean_squared_log_error(y_test,pred_y))
    print ("Err value: %.2f" % err)

In [9]:
from sklearn.neural_network import MLPRegressor

model= MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
        beta_2=0.999, early_stopping=False, epsilon=1e-05,
        hidden_layer_sizes=100, learning_rate='adaptive',
        learning_rate_init=0.001, max_iter=1000, momentum=0.9,
        nesterovs_momentum=True, power_t=0.5, random_state=None,
        shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
        verbose=1, warm_start=False)

model.fit(X_train, y_train)
eval(model, X_test, y_test)

Iteration 1, loss = 1.13982671
Iteration 2, loss = 1.12838448
Iteration 3, loss = 1.12342176
Iteration 4, loss = 1.12081496
Iteration 5, loss = 1.11807852
Iteration 6, loss = 1.11681666
Iteration 7, loss = 1.11305320
Iteration 8, loss = 1.11159766
Iteration 9, loss = 1.10974995
Iteration 10, loss = 1.10761086
Iteration 11, loss = 1.10690111
Iteration 12, loss = 1.10310697
Iteration 13, loss = 1.10178871
Iteration 14, loss = 1.09986045
Iteration 15, loss = 1.09744277
Iteration 16, loss = 1.09644452
Iteration 17, loss = 1.09376086
Iteration 18, loss = 1.09165472
Iteration 19, loss = 1.08876273
Iteration 20, loss = 1.08678679
Iteration 21, loss = 1.08516723
Iteration 22, loss = 1.08214389
Iteration 23, loss = 1.07888895
Iteration 24, loss = 1.07811967
Iteration 25, loss = 1.07657163
Iteration 26, loss = 1.07453788
Iteration 27, loss = 1.07177539
Iteration 28, loss = 1.07003501
Iteration 29, loss = 1.06634469
Iteration 30, loss = 1.06498522
Iteration 31, loss = 1.06418562
Iteration 32, los

Iteration 253, loss = 0.86815826
Iteration 254, loss = 0.86767301
Iteration 255, loss = 0.86417597
Iteration 256, loss = 0.86680442
Iteration 257, loss = 0.86488116
Iteration 258, loss = 0.86649564
Iteration 259, loss = 0.86385271
Iteration 260, loss = 0.86762527
Iteration 261, loss = 0.86433308
Iteration 262, loss = 0.86527541
Iteration 263, loss = 0.86309807
Iteration 264, loss = 0.86469732
Iteration 265, loss = 0.86660180
Iteration 266, loss = 0.86209700
Iteration 267, loss = 0.86297067
Iteration 268, loss = 0.86114144
Iteration 269, loss = 0.86342301
Iteration 270, loss = 0.86163690
Iteration 271, loss = 0.86122504
Iteration 272, loss = 0.86314920
Iteration 273, loss = 0.86073691
Iteration 274, loss = 0.86054938
Iteration 275, loss = 0.85986928
Iteration 276, loss = 0.85829718
Iteration 277, loss = 0.85963019
Iteration 278, loss = 0.85797602
Iteration 279, loss = 0.86216969
Iteration 280, loss = 0.85798264
Iteration 281, loss = 0.85807658
Iteration 282, loss = 0.85735920
Iteration 

In [20]:
from sklearn import metrics
param_grid = {'hidden_layer_sizes': [50,100],
               'activation': ['relu'],
               'solver': ['adam'],
               'learning_rate': ['constant'],
               'learning_rate_init': [0.001],
               'power_t': [0.5],
               'alpha': [0.0001],
               'max_iter': [1000],
               'early_stopping': [True, False],
               'warm_start': [True,False]}
_GSmodel = GridSearchCV(model, param_grid=param_grid,
                    cv=3, verbose=True, pre_dispatch='2*n_jobs')


_GSmodel.fit(X_train, y_train)

#eval(best_model, X_test, y_test)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Streaming output truncated to the last 5000 lines.
Iteration 5, loss = 1.13670745
Validation score: 0.049586
Iteration 6, loss = 1.13594194
Validation score: 0.054053
Iteration 7, loss = 1.13233229
Validation score: 0.054696
Iteration 8, loss = 1.13013157
Validation score: 0.054479
Iteration 9, loss = 1.12890550
Validation score: 0.056987
Iteration 10, loss = 1.12739146
Validation score: 0.059423
Iteration 11, loss = 1.12471725
Validation score: 0.056755
Iteration 12, loss = 1.12505245
Validation score: 0.060183
Iteration 13, loss = 1.12118934
Validation score: 0.055810
Iteration 14, loss = 1.12140721
Validation score: 0.059910
Iteration 15, loss = 1.11849608
Validation score: 0.051515
Iteration 16, loss = 1.11760346
Validation score: 0.050761
Iteration 17, loss = 1.11601610
Validation score: 0.059373
Iteration 18, loss = 1.11485113
Validation score: 0.055742
Iteration 19, loss = 1.11312377
Validation score: 0.052247
Iteration 20, loss = 1.11196382
Validation score: 0.057759
Iteration 

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 38.6min finished


Iteration 1, loss = 1.13984122
Validation score: 0.063864
Iteration 2, loss = 1.12631864
Validation score: 0.060596
Iteration 3, loss = 1.12279349
Validation score: 0.066121
Iteration 4, loss = 1.11929725
Validation score: 0.062408
Iteration 5, loss = 1.11757347
Validation score: 0.065063
Iteration 6, loss = 1.11453769
Validation score: 0.066468
Iteration 7, loss = 1.11325223
Validation score: 0.067156
Iteration 8, loss = 1.11062586
Validation score: 0.066793
Iteration 9, loss = 1.10921260
Validation score: 0.064876
Iteration 10, loss = 1.10726248
Validation score: 0.066735
Iteration 11, loss = 1.10576828
Validation score: 0.065685
Iteration 12, loss = 1.10417944
Validation score: 0.064210
Iteration 13, loss = 1.10195221
Validation score: 0.063603
Iteration 14, loss = 1.10089471
Validation score: 0.064261
Iteration 15, loss = 1.09867469
Validation score: 0.064704
Iteration 16, loss = 1.09820638
Validation score: 0.061412
Iteration 17, loss = 1.09496721
Validation score: 0.061135
Iterat

GridSearchCV(cv=3, error_score=nan,
             estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                    batch_size='auto', beta_1=0.9, beta_2=0.999,
                                    early_stopping=False, epsilon=1e-08,
                                    hidden_layer_sizes=100,
                                    learning_rate='adaptive',
                                    learning_rate_init=0.001, max_fun=15000,
                                    max_iter=1000, momentum=0.9,
                                    n_iter_no_change=10,
                                    nesterovs_momentum=True, power_t=0.5,
                                    random_state=No...
             iid='deprecated', n_jobs=None,
             param_grid={'activation': ['relu'], 'alpha': [0.0001],
                         'early_stopping': [True, False],
                         'hidden_layer_sizes': [50, 100],
                         'learning_rate': ['constant'],
       

In [21]:
_GSmodel.best_params_, _GSmodel.best_estimator_

({'activation': 'relu',
  'alpha': 0.0001,
  'early_stopping': True,
  'hidden_layer_sizes': 50,
  'learning_rate': 'constant',
  'learning_rate_init': 0.001,
  'max_iter': 1000,
  'power_t': 0.5,
  'solver': 'adam',
  'warm_start': True},
 MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=50, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=True))

In [10]:
model =  MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=7, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=1000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=1, warm_start=True)

model.fit(X_train, y_train)
eval(model, X_test, y_test)

Iteration 1, loss = 1.16936019
Validation score: 0.059998
Iteration 2, loss = 1.13501358
Validation score: 0.066477
Iteration 3, loss = 1.13037800
Validation score: 0.068838
Iteration 4, loss = 1.12806728
Validation score: 0.070655
Iteration 5, loss = 1.12599398
Validation score: 0.071391
Iteration 6, loss = 1.12436157
Validation score: 0.070693
Iteration 7, loss = 1.12282264
Validation score: 0.072917
Iteration 8, loss = 1.12211774
Validation score: 0.072513
Iteration 9, loss = 1.12102164
Validation score: 0.070368
Iteration 10, loss = 1.11988256
Validation score: 0.070656
Iteration 11, loss = 1.11901989
Validation score: 0.073564
Iteration 12, loss = 1.11805667
Validation score: 0.072233
Iteration 13, loss = 1.11743996
Validation score: 0.073602
Iteration 14, loss = 1.11648328
Validation score: 0.074754
Iteration 15, loss = 1.11617035
Validation score: 0.074024
Iteration 16, loss = 1.11489232
Validation score: 0.074450
Iteration 17, loss = 1.11417842
Validation score: 0.070758
Iterat

In [ ]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']

X_submit = scaler.transform(X_submit)

In [ ]:
eval(model, X_submit, y_submit)

RMSLE: 0.54
R2 value: 0.02
Err value: 0.54


In [ ]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
pred_y = model.predict(X_submit)

submission = pd.DataFrame()
submission['MemberID'] = data['MemberID']

pred_y = pred_y.clip(min=0)
pred_y = pred_y.clip(max=15)
# pred_y = pred_y.astype(int)

submission['DIH'] = pred_y
submission.groupby(['DIH']).size()
submission.to_csv('../5.ensemble/mlp.csv',index=False)

In [ ]:
submission[submission['DIH']>=7]

,MemberID,DIH
16966,17124817,8.246692


In [ ]:
real = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')


In [ ]:
real[real['MemberID']==17124817]

,MemberID,Year,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
16966,17124817,Y2,1,10.0,1,0,0,0,0,0,...,7,1,5.181818,11,5,5,5.0,1,0,0


In [26]:
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def dSigmoid(Z):
    s = 1/(1+np.exp(-Z))
    dZ = s * (1-s)
    return dZ

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(Y_hat, Y):
    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[0]
    L = -(1/m) * L_sum
    return L

def rmsle(y_h, y):
        #y = y.argmax(1) 
        # The root mean square logarithmic error
        y_h = y_h.clip(min=0)
        y_h = y_h.clip(max=15)
        y_h = y_h.round(0).astype(int)
        msle = np.mean( (np.log(y_h + 1) - np.log(y + 1)) ** 2)
        rmsle = math.sqrt(msle)
        return rmsle

def Relu(Z):
    return np.maximum(0,Z)

def dRelu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

class NeuralNetwork:
    def __init__(self, x, y, op_dim = 1, hidden_size = 100, learning_rate=0.5):
        self.x = x
        hd_dim = hidden_size # Number of neurons in hidden layer
        self.lr = learning_rate
        ip_dim = x.shape[1] # Number of input features
        if(op_dim !=1):
          self.op_dim = y.shape[1] # Number of output classes
        else:
          self.op_dim = 1  
        print(op_dim)
        # Initializing the parameters
        self.w1 = np.random.randn(ip_dim, hd_dim)
        self.b1 = np.zeros((1, hd_dim))
        self.w2 = np.random.randn(hd_dim, self.op_dim)
        self.b2 = np.zeros((1, self.op_dim))
        self.y = y
        
        self.loss_history = []

    def feedforward(self):
        self.z1 = np.matmul(self.x, self.w1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.a1 = Relu(self.z1)
        self.z2 = np.matmul(self.a1, self.w2) + self.b2
        self.a2 = softmax(self.z2)
        
    def backprop(self):
        # Compute the loss value
        #loss_value = cross_entropy(self.a2, self.y)
        loss_value = rmsle(self.a2, self.y)
        self.loss_history.append(loss_value)
        
        # Backpropagation
        m = self.x.shape[0]
        dZ2 = self.a2- self.y
        dW2 = (1./m) * np.matmul(self.a1.T, dZ2)
        db2 = (1./m) * np.sum(dZ2, axis=0, keepdims=True)

        dA1 = np.matmul(dZ2, self.w2.T)
        #dZ1 = dA1 * dRelu(self.z1)
        dZ1 = dA1 * dSigmoid(self.z1) 
        dW1 = (1./m) * np.matmul(self.x.T, dZ1)
        db1 = (1./m) * np.sum(dZ1, axis=0, keepdims=True)

        # Update parameters
        self.w2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.w1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def predict(self, X_pred):
        temp = self.x
        self.x = X_pred
        self.feedforward()
        self.x = temp
        return self.a2
    
    def get_acc(self, X_test, y_test):
        y_pred = self.predict(X_test).argmax(1)
        y_test = y_test.argmax(1)
        acc = np.sum(y_pred == y_test)/len(y_pred)
        return acc

    def rmsle(self, X_test, y_test):
        y_pred = self.predict(X_test).argmax(1)
        y_test = y_test.argmax(1) 
        # The root mean square logarithmic error
        y_pred = y_pred.clip(min=0)
        y_pred = y_pred.clip(max=15)
        y_pred = y_pred.round(0).astype(int)
        msle = np.mean( (np.log(y_pred + 1) - np.log(y_test + 1)) ** 2)
        rmsle = math.sqrt(msle)
        return rmsle

In [14]:
X_train.shape

(103231, 137)

In [27]:
import time
learning_rate = 0.01


print("learning rate: " + str(learning_rate))
print("hidden_size: 7")

model = NeuralNetwork(X_train, y_train, op_dim = 1, hidden_size=7, learning_rate=learning_rate)

epochs = 1000
acc_list = []
rmsle_list = []
start = time.time()
for i in range(epochs):
    model.feedforward()
    model.backprop()
    acc = model.get_acc(X_train, y_train), model.get_acc(X_test, y_test)
    acc_list.append(acc)
    rmsle = model.rmsle(X_train, y_train), model.rmsle(X_test, y_test)
    rmsle_list.append(rmsle)
    if (i+1)%10 == 0:
        print('Epoch {} -- Loss: {:.4f} -- RMSLE:{:.4f} '.format(i+1, model.loss_history[-1], rmsle))
done = time.time()

elapsed = done - start 
print("time:" + str(elapsed))

learning rate: 0.01
hidden_size: 7
1


MemoryError: Unable to allocate 79.4 GiB for an array with shape (103231, 103231) and data type float64